# Monte Carlo Pricer: Performance & Convergence Analysis

This notebook demonstrates the performance characteristics and convergence behavior of the Monte Carlo option pricer with:
- Multi-threading optimization
- SIMD batch processing
- Variance reduction techniques

## Setup Instructions

Before running this notebook, install the module:
```bash
cd c:\CPP\montecarlo-pricer
pip install -e .
```

If you're running this for the first time, the build may take a minute or two.

In [ ]:
# Ensure the module is installed
import subprocess
import sys

try:
    import montecarlo_pricer
    print("✓ montecarlo_pricer module found")
except ImportError:
    print("Building and installing montecarlo_pricer module...")
    result = subprocess.run(
        [sys.executable, "-m", "pip", "install", "-e", "c:\\CPP\\montecarlo-pricer", "--force-reinstall", "--no-deps"],
        capture_output=True,
        text=True,
        timeout=300
    )
    if result.returncode != 0:
        print("STDOUT:", result.stdout)
        print("STDERR:", result.stderr)
        raise RuntimeError(f"Failed to install module: {result.stderr}")
    print("✓ Module installed successfully")
    import montecarlo_pricer
    print("✓ montecarlo_pricer module loaded")

In [4]:
# Import required libraries
import montecarlo_pricer as mcp
import numpy as np
import matplotlib.pyplot as plt
import time
from collections import defaultdict

# Set matplotlib style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

Building and installing montecarlo_pricer module...
✓ Module installed successfully
✓ montecarlo_pricer module loaded


## Setup: Create Pricer and Configuration

In [ ]:
# Initialize pricer
pricer = mcp.MonteCarloPricer(seed=42)

# Base configuration
config = mcp.PricingConfig()
config.S0 = 100.0
config.K = 100.0
config.r = 0.05
config.sigma = 0.2
config.T = 1.0
config.option_type = "call"
config.use_antithetic = True

# Get analytical price for reference
analytical_price = pricer.analytical_price(config)
print(f"Black-Scholes Analytical Price: {analytical_price:.6f}")

## Plot 1: Error vs Paths (Log-Log)

Shows how pricing error decreases as the number of Monte Carlo paths increases. Demonstrates convergence rate of O(1/√N).

In [ ]:
# Test different path counts
path_counts = [1_000, 5_000, 10_000, 50_000, 100_000, 500_000, 1_000_000, 5_000_000]
errors = []
std_errors = []

config.n_threads = 8  # Use parallel for speed

print("Computing convergence data...")
for n_paths in path_counts:
    config.n_paths = n_paths
    result = pricer.price_mc_parallel(config)
    
    error = abs(result.price - analytical_price)
    errors.append(error)
    std_errors.append(result.std_error)
    
    print(f"Paths: {n_paths:>9,} | Error: {error:.6f} | Std Error: {result.std_error:.6f}")

# Create log-log plot
fig, ax = plt.subplots(figsize=(10, 7))

ax.loglog(path_counts, errors, 'o-', linewidth=2, markersize=8, label='Pricing Error', color='#e74c3c')
ax.loglog(path_counts, std_errors, 's-', linewidth=2, markersize=8, label='Standard Error', color='#3498db')

# Add theoretical O(1/√N) reference line
theoretical = [errors[0] * np.sqrt(path_counts[0] / n) for n in path_counts]
ax.loglog(path_counts, theoretical, '--', linewidth=1.5, label='O(1/√N) Reference', color='gray', alpha=0.7)

ax.set_xlabel('Number of Paths', fontsize=13, fontweight='bold')
ax.set_ylabel('Error', fontsize=13, fontweight='bold')
ax.set_title('Convergence: Error vs Number of Paths (Log-Log)', fontsize=15, fontweight='bold', pad=20)
ax.legend(fontsize=11, loc='best')
ax.grid(True, which="both", ls="-", alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal error at {path_counts[-1]:,} paths: {errors[-1]:.6f}")
print(f"Error reduction from {path_counts[0]:,} to {path_counts[-1]:,} paths: {errors[0]/errors[-1]:.2f}x")

## Plot 2: Runtime vs Threads

Measures execution time as a function of thread count to identify optimal parallelization.

In [ ]:
# Test different thread counts
import os
max_threads = os.cpu_count() or 8
thread_counts = [1, 2, 4, 8]
if max_threads >= 16:
    thread_counts.extend([16, max_threads])
elif max_threads > 8:
    thread_counts.append(max_threads)

runtimes = []
config.n_paths = 10_000_000  # Fixed path count for fair comparison

print("Benchmarking threading performance...")
for n_threads in thread_counts:
    config.n_threads = n_threads
    
    start = time.perf_counter()
    result = pricer.price_mc_parallel(config)
    elapsed = time.perf_counter() - start
    
    runtimes.append(elapsed)
    throughput = config.n_paths / elapsed
    
    print(f"Threads: {n_threads:>2} | Time: {elapsed:.3f}s | Throughput: {throughput/1e6:.1f}M paths/sec")

# Create runtime plot
fig, ax = plt.subplots(figsize=(10, 7))

ax.plot(thread_counts, runtimes, 'o-', linewidth=2.5, markersize=10, color='#9b59b6')

# Annotate points with values
for i, (threads, runtime) in enumerate(zip(thread_counts, runtimes)):
    ax.annotate(f'{runtime:.2f}s', 
                xy=(threads, runtime), 
                xytext=(0, 10), 
                textcoords='offset points',
                ha='center',
                fontsize=10,
                bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.3))

ax.set_xlabel('Number of Threads', fontsize=13, fontweight='bold')
ax.set_ylabel('Runtime (seconds)', fontsize=13, fontweight='bold')
ax.set_title(f'Runtime vs Thread Count ({config.n_paths:,} paths)', fontsize=15, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3)
ax.set_xticks(thread_counts)

plt.tight_layout()
plt.show()

print(f"\nBest runtime: {min(runtimes):.3f}s with {thread_counts[runtimes.index(min(runtimes))]} threads")
print(f"Speedup from 1 to {thread_counts[-1]} threads: {runtimes[0]/runtimes[-1]:.2f}x")

## Plot 3: Speedup Curve

Shows parallel speedup relative to single-threaded execution. Ideal (linear) speedup and actual speedup with efficiency metrics.

In [ ]:
# Calculate speedup and efficiency
base_time = runtimes[0]
speedups = [base_time / t for t in runtimes]
efficiencies = [speedup / threads * 100 for speedup, threads in zip(speedups, thread_counts)]

# Create speedup plot with dual y-axis
fig, ax1 = plt.subplots(figsize=(12, 7))

# Speedup on left y-axis
color1 = '#e67e22'
ax1.plot(thread_counts, speedups, 'o-', linewidth=2.5, markersize=10, 
         color=color1, label='Actual Speedup')
ax1.plot(thread_counts, thread_counts, '--', linewidth=2, 
         color='gray', alpha=0.5, label='Ideal (Linear) Speedup')
ax1.set_xlabel('Number of Threads', fontsize=13, fontweight='bold')
ax1.set_ylabel('Speedup Factor', fontsize=13, fontweight='bold', color=color1)
ax1.tick_params(axis='y', labelcolor=color1)
ax1.set_xticks(thread_counts)
ax1.grid(True, alpha=0.3)

# Efficiency on right y-axis
ax2 = ax1.twinx()
color2 = '#27ae60'
ax2.plot(thread_counts, efficiencies, 's-', linewidth=2.5, markersize=10, 
         color=color2, label='Parallel Efficiency')
ax2.set_ylabel('Parallel Efficiency (%)', fontsize=13, fontweight='bold', color=color2)
ax2.tick_params(axis='y', labelcolor=color2)
ax2.set_ylim([0, 110])

# Add annotations
for i, (threads, speedup, eff) in enumerate(zip(thread_counts, speedups, efficiencies)):
    ax1.annotate(f'{speedup:.1f}x', 
                xy=(threads, speedup), 
                xytext=(-15, -15), 
                textcoords='offset points',
                fontsize=9,
                color=color1,
                fontweight='bold')

ax1.set_title('Parallel Speedup and Efficiency', fontsize=15, fontweight='bold', pad=20)

# Combine legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=11)

plt.tight_layout()
plt.show()

print("\nSpeedup Summary:")
for threads, speedup, eff in zip(thread_counts, speedups, efficiencies):
    print(f"  {threads:>2} threads: {speedup:>5.2f}x speedup, {eff:>5.1f}% efficiency")

## Plot 4: Standard Error vs Variance Reduction Technique

Compares the effectiveness of different variance reduction methods in reducing estimation error.

In [ ]:
# Test variance reduction techniques
config.n_paths = 1_000_000
config.n_threads = 8

techniques = [
    ("Standard MC", False, False),
    ("Antithetic\nVariates", True, False),
    ("Control Variate\n(β=1, Sanity Check)", False, True),
    ("Antithetic\n+ Control", True, True),
]

technique_names = []
std_errors_vr = []
prices = []
vr_factors = []

print("Testing variance reduction techniques...\n")
base_std_error = None

for name, use_anti, use_cv in techniques:
    config.use_antithetic = use_anti
    config.use_control_variate = use_cv
    
    result = pricer.price_mc_parallel(config)
    
    technique_names.append(name)
    std_errors_vr.append(result.std_error)
    prices.append(result.price)
    
    if base_std_error is None:
        base_std_error = result.std_error
        vr_factor = 1.0
    else:
        vr_factor = base_std_error / result.std_error
    
    vr_factors.append(vr_factor)
    
    print(f"{name:<30} Std Error: {result.std_error:.6f}, VR Factor: {vr_factor:.2f}x")
    if use_cv and result.control_variate_used:
        # Check if control variate diagnostics are available (requires rebuilt module)
        if hasattr(result, 'control_beta') and hasattr(result, 'variance_reduction_factor'):
            print(f"  {'└─ Control:':<28} β={result.control_beta:.2f}, Var.Red.={result.variance_reduction_factor:.2f}x")
        else:
            print(f"  {'└─ Control:':<28} Enabled (rebuild module for diagnostics)")

# Create bar plot for standard errors
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Plot 1: Standard Error Comparison
colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12']
bars1 = ax1.bar(range(len(technique_names)), std_errors_vr, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)

# Add value labels on bars
for i, (bar, value) in enumerate(zip(bars1, std_errors_vr)):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'{value:.6f}',
            ha='center', va='bottom', fontsize=10, fontweight='bold')

ax1.set_ylabel('Standard Error', fontsize=13, fontweight='bold')
ax1.set_title('Standard Error by Variance Reduction Technique', fontsize=14, fontweight='bold', pad=15)
ax1.set_xticks(range(len(technique_names)))
ax1.set_xticklabels(technique_names, fontsize=10)
ax1.grid(True, axis='y', alpha=0.3)

# Plot 2: Variance Reduction Factor
bars2 = ax2.bar(range(len(technique_names)), vr_factors, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)

# Add value labels on bars
for i, (bar, value) in enumerate(zip(bars2, vr_factors)):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{value:.2f}x',
            ha='center', va='bottom', fontsize=10, fontweight='bold')

# Add reference line at 1.0
ax2.axhline(y=1.0, color='gray', linestyle='--', linewidth=2, alpha=0.5, label='Baseline')

ax2.set_ylabel('Variance Reduction Factor', fontsize=13, fontweight='bold')
ax2.set_title('Variance Reduction Effectiveness', fontsize=14, fontweight='bold', pad=15)
ax2.set_xticks(range(len(technique_names)))
ax2.set_xticklabels(technique_names, fontsize=10)
ax2.grid(True, axis='y', alpha=0.3)
ax2.legend(fontsize=10)

plt.tight_layout()
plt.show()

print(f"\nBest technique: {technique_names[std_errors_vr.index(min(std_errors_vr))]} with {min(std_errors_vr):.6f} std error")
print(f"Maximum variance reduction: {max(vr_factors):.2f}x")

## Summary: Combined Performance Overview

In [ ]:
print("=" * 80)
print("PERFORMANCE SUMMARY")
print("=" * 80)

print(f"\n>>> Configuration:")
print(f"  Stock Price (S0): ${config.S0:.2f}")
print(f"  Strike (K): ${config.K:.2f}")
print(f"  Volatility (σ): {config.sigma*100:.0f}%")
print(f"  Risk-free Rate (r): {config.r*100:.1f}%")
print(f"  Time to Maturity (T): {config.T:.1f} year")
print(f"  Black-Scholes Price: ${analytical_price:.6f}")

print(f"\n>>> Convergence:")
print(f"  Converges at O(1/√N) rate")
print(f"  Error at 1M paths: {errors[path_counts.index(1_000_000)]:.6f}")
print(f"  Error at 5M paths: {errors[-1]:.6f}")

print(f"\n>>> Threading Performance:")
print(f"  Single-threaded: {runtimes[0]:.3f}s")
print(f"  Best multi-threaded: {min(runtimes):.3f}s ({thread_counts[runtimes.index(min(runtimes))]} threads)")
print(f"  Maximum speedup: {max(speedups):.2f}x")
print(f"  Peak efficiency: {max(efficiencies):.1f}%")

print(f"\n>>> Variance Reduction:")
print(f"  Standard MC std error: {std_errors_vr[0]:.6f}")
print(f"  Best technique: {technique_names[std_errors_vr.index(min(std_errors_vr))]}")
print(f"  Best std error: {min(std_errors_vr):.6f}")
print(f"  Maximum improvement: {max(vr_factors):.2f}x")

print("\n" + "=" * 80)